In [1]:
mkdir model


In [7]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Example: Saving a PyTorch model
import torch
torch.save(model.state_dict(), 'model_epoch.pt')


NameError: name 'model' is not defined

In [11]:
import tarfile
import boto3
import os

# Download the file from S3
s3_bucket = 'midtermproject-2'
s3_key = 'model_epoch_10.pt'
local_file_path = 'model_epoch_10.pt'

s3_client = boto3.client('s3')
s3_client.download_file(s3_bucket, s3_key, local_file_path)

# Create a tar.gz archive
with tarfile.open('model_epoch.tar.gz', 'w:gz') as f:
    f.add(local_file_path)

# Cleanup: Remove the downloaded file
os.remove(local_file_path)


In [15]:
import boto3

# Create an S3 client
s3 = boto3.client('s3')
s3.upload_file('model_epoch.tar.gz', 'midtermproject-2', 'model_epoch.tar.gz')


In [17]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import io
import boto3
import json
from PIL import Image
import tarfile

# Load the model
model = models.resnet50(pretrained=False)
num_classes = 10  # Example: Modify based on your model architecture
model.fc = nn.Linear(2048, num_classes)  # Example: Modify based on your model architecture

def transform_image(image_bytes):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(io.BytesIO(image_bytes))
    return transform(image).unsqueeze(0)

def predict_fn(input_data, model):
    with torch.no_grad():
        input_tensor = transform_image(input_data)
        output = model(input_tensor)
        return output.numpy().tolist()

def model_fn(model_dir):
    with tarfile.open(f'{model_dir}/model_epoch.tar.gz', 'r:gz') as tar:
        tar.extractall()
    model.load_state_dict(torch.load('pytorch_model.pth', map_location=torch.device('cpu')))
    model.eval()
    return model

# Endpoint handler
def predict(event, context):
    try:
        request_body = json.loads(event['body'])
        input_data = request_body['data']
        prediction = predict_fn(input_data, model)
        return {
            'statusCode': 200,
            'body': json.dumps(prediction)
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps(str(e))
        }

# Upload the model file to S3
s3 = boto3.client('s3')
s3.upload_file('model_epoch.tar.gz', 'midtermproject-2', 'model_epoch.tar.gz')


In [20]:
import sagemaker

# Define the IAM role
role = sagemaker.get_execution_role()


In [21]:
role = 'arn:aws:iam::533267305253:role/LabRole'


In [ ]:
import sagemaker

# Define the IAM role
role = sagemaker.get_execution_role()

from sagemaker.pytorch import PyTorchModel

# Upload the model archive to S3
s3.upload_file('model_epoch.tar.gz', 'midtermproject-2', 'model_epoch.tar.gz')

# Create a PyTorchModel with framework_version and py_version specified
pytorch_model = PyTorchModel(model_data='s3://midtermproject-2/model_epoch.tar.gz',
                             role=role,
                             framework_version='1.8.1',
                             py_version='py3')

# Deploy the model
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1)


---